<a href="https://colab.research.google.com/github/stefanoiervese/DL_Project/blob/main/DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.getcwd()
path='/content/drive/MyDrive/mathematics_dataset-v1.0/train-easy/arithmetic__add_or_sub.txt'
#os.listdir(path)
with open(path, "r") as file:
    # Leggi il contenuto del file
    content = file.read()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import random

data_list = [x for x in content.split('\n')]
data_list=data_list[:-1]
len_data=len(data_list)
quest=[]
ans=[]
for i in range(len_data):
  if(i%2==0):
    quest.append(data_list[i])
  else:
    ans.append(data_list[i])
coppie = list(zip(quest,ans))
random.shuffle(coppie)
quest, ans=zip(*coppie)
l=int(len(quest)/3)
train_q=quest[:2*l]
test_q=quest[2*l:]
train_a=ans[:2*l]
test_a=ans[2*l:]

voc_size=len([sentence.split() for sentence in data_list]) #1333332


type(train_q)


tuple

# Token

In [3]:

import nltk
import re
from nltk.corpus import words
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [4]:
class Tokenizer:
    def __init__(self, vocab):
        self.vocab = vocab
        self.word_to_id = {word: idx for idx, word in enumerate(vocab)}

    def tokenize(self, text):
        tokens = re.findall(r'\d|\.|[-=+*/()?]|\S+', text)



        token_id = []
        unknown_tokens = []

        new_tokens = []
        for token in tokens:
          flag=0
          if '.' in token and token != '.':
            token=token=token.split('.')[0]
            flag=1
          if token in self.word_to_id:
            token_id.append(self.word_to_id[token])
          else:
                token_id.append(self.word_to_id['unknown'])
                unknown_tokens.append(token)
          if(flag==1):
            token_id.append(self.word_to_id['.'])


        if unknown_tokens:
            print("Parole sconosciute:", unknown_tokens)

        return token_id


math_vocab = ['0','1','2','3','4','5','6','7','8','9','.','+',',','-','*','/', '?','(',')']
english_words = words.words()
vocab = math_vocab+english_words
tokenizer = Tokenizer(vocab)

text = "What is 3?"
token_ids = tokenizer.tokenize(text)
print("Token IDs:", token_ids)



Parole sconosciute: ['What']
Token IDs: [218836, 98436, 3, 16]


In [ ]:
len(vocab)

# DataLoader

In [6]:
import torch
qt=[]
at=[]
for x in train_q:
  qt.append(torch.tensor(tokenizer.tokenize(x.lower())))
for x in train_a:
  at.append(torch.tensor(tokenizer.tokenize(x.lower())))








In [ ]:
qt[0]

In [7]:

import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pack_sequence, pad_sequence


class Dataset(Dataset):
    def __init__(self, questions, answers):
        self.questions = questions
        self.answers = answers

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        return self.questions[idx], self.answers[idx]


voc_size=100
src_data = torch.randint(1, voc_size, (3264, 8))
tg_data=torch.randint(1,voc_size,(3264,8))
fake_dataset=Dataset(src_data,tg_data)
fake_loader=DataLoader(fake_dataset,batch_size=32,shuffle=True)

batch_size = 32
train_dataset = Dataset(qt,at)
#train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,collate_fn=lambda batch: (
    pad_sequence([item[0] for item in batch], batch_first=True),
    pad_sequence([item[1] for item in batch], batch_first=True)
))
#test_dataset = Dataset(test_q,test_a)
#test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=True)


In [ ]:
type(qt)

In [ ]:
src_data.shape

# Transformer

In [8]:

import torch
import torch.nn as nn
import math
import pytorch_lightning as pl
import torch.optim as optim

class MultiHeadAttention(pl.LightningModule):
    def __init__(self, emb_dim, num_heads,dropout=0.1):
        super(MultiHeadAttention, self).__init__()
        assert emb_dim % num_heads == 0, "emb_dim must be divisible by num_heads"

        self.emb_dim = emb_dim
        self.num_heads = num_heads
        self.head_dim = self.emb_dim // num_heads

        # Inizializzazione dei moduli lineari per proiettare Q, K, V e l'output.
        self.W_q = nn.Linear(self.emb_dim, self.emb_dim)
        self.W_k = nn.Linear(self.emb_dim, self.emb_dim)
        self.W_v = nn.Linear(self.emb_dim, self.emb_dim)
        self.W_o = nn.Linear(self.emb_dim, self.emb_dim)


    def forward(self, q,k,v, mask=None):
        q = self.split_heads(self.W_q(q))
        k = self.split_heads(self.W_k(k))
        v = self.split_heads(self.W_v(v))

        att = self.att_score(q, k, v, mask)
        out = self.W_o(self.combine_heads(att))

        return out

    def att_score(self, q, k, v, mask=None):
        attn_scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.head_dim)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, v)
        return output

    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.head_dim).transpose(1, 2)

    def combine_heads(self, x):
        batch_size, _, seq_length, head_dim = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.emb_dim)







class EncoderLayer(pl.LightningModule):
    def __init__(self, emb_dim, num_heads, feedforward_dim=32, dropout=0.1):
        super(EncoderLayer, self).__init__()

        self.multihead_attention = MultiHeadAttention(emb_dim, num_heads)

        self.feedforward = nn.Sequential(
            nn.Linear(emb_dim, feedforward_dim),
            nn.ReLU(),
            nn.Linear(feedforward_dim, emb_dim)
        )

        self.layer_norm1 = nn.LayerNorm(emb_dim)
        self.layer_norm2 = nn.LayerNorm(emb_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):

        att = self.multihead_attention(x,x,x, mask)

        add_nor = self.layer_norm1(x + self.dropout(att))
        ff_out = self.feedforward(add_nor)
        out = self.layer_norm2(add_nor + self.dropout(ff_out))


        return out





class Encoder(pl.LightningModule):
    def __init__(self, emb_dim, num_heads, num_layers=6, feedforward_dim=32, dropout=0.1):
        super(Encoder, self).__init__()

        self.layers = nn.ModuleList([
            EncoderLayer(emb_dim, num_heads, feedforward_dim, dropout)
            for _ in range(num_layers)
        ])

    def forward(self, x, mask=None):
        for layer in self.layers:
            x = layer(x, mask)
        return x


class DecoderLayer(pl.LightningModule):
    def __init__(self, emb_dim, num_heads, feedforward_dim=32, dropout=0.1):
        super(DecoderLayer, self).__init__()

        # Self-Attention Layer (Auto-Attention)
        self.self_attention = MultiHeadAttention(emb_dim, num_heads)

        # Cross-Attention Layer (Attenzione incrociata con l'encoder)
        self.cross_attention = MultiHeadAttention(emb_dim, num_heads)

        # Feedforward Neural Network Layer
        self.feedforward = nn.Sequential(
            nn.Linear(emb_dim, feedforward_dim),
            nn.ReLU(),
            nn.Linear(feedforward_dim, emb_dim)
        )

        # Layer Normalization
        self.layer_norm1 = nn.LayerNorm(emb_dim)
        self.layer_norm2 = nn.LayerNorm(emb_dim)
        self.layer_norm3 = nn.LayerNorm(emb_dim)

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, encoder_output, src_mask=None, tgt_mask=None):

        attn_output = self.self_attention(x, x, x, tgt_mask)
        out1 = self.layer_norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attention(out1, encoder_output, encoder_output, src_mask)
        out2 = self.layer_norm2(out1 + self.dropout(attn_output))
        ff_output = self.feedforward(x)
        out3 = self.layer_norm3(out2 + self.dropout(ff_output))
        return out3

class Decoder(pl.LightningModule):
    def __init__(self, emb_dim, num_heads, num_layers=6, feedforward_dim=32, dropout=0.1):
        super(Decoder, self).__init__()


        self.layers = nn.ModuleList([
            DecoderLayer(emb_dim, num_heads, feedforward_dim, dropout)
            for _ in range(num_layers)
        ])

    def forward(self, x, encoder_output, src_mask=None, tgt_mask=None):
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)
        return x

import pytorch_lightning as pl
import torch
import torch.nn as nn

class Transformer(pl.LightningModule):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model=4, num_heads=2, num_layers=6, d_ff=256, max_seq_length=8, dropout=0.1):
        super(Transformer, self).__init__()
        self.src_embedding = nn.Embedding(src_vocab_size, d_model)
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, d_model)

        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model, num_heads, d_ff, dropout),
            num_layers
        )
        self.transformer_decoder = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(d_model, num_heads, d_ff, dropout),
            num_layers
        )

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, tgt):
        src_embedded = self.dropout(self.src_embedding(src))
        tgt_embedded = self.dropout(self.tgt_embedding(tgt))

        #src_mask = self.generate_padding_mask(src)
        #tgt_mask = self.generate_square_subsequent_mask(tgt.size(1))
        src_mask=None
        tgt_mask=None

        enc_output = self.transformer_encoder(src_embedded, src_key_padding_mask=src_mask)
        dec_output = self.transformer_decoder(tgt_embedded, enc_output, tgt_mask=tgt_mask)

        output = self.fc(dec_output)
        return output

    def generate_padding_mask(self, src):
        src_mask = (src == 0)
        return src_mask

    def generate_square_subsequent_mask(self, size):
        mask = (torch.triu(torch.ones(size, size)) == 0).transpose(0, 1)
        return mask.float()

    def training_step(self, batch, batch_idx):
        src, tgt = batch
        output = self(src, tgt)
        tgt_mask = (tgt != 0)
        loss = nn.CrossEntropyLoss(ignore_index=0)(output.view(-1, output.size(-1)), tgt.view(-1))
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        src, tgt = batch
        output = self(src, tgt)
        tgt_mask = (tgt != 0)
        loss = nn.CrossEntropyLoss(ignore_index=0)(output.view(-1, output.size(-1)), tgt.view(-1))
        self.log('val_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-4)
        return optimizer





voc_len=len(vocab)
t=Transformer(voc_len,voc_len)


t()
# Addestra il modello
trainer = pl.Trainer(max_epochs=10)  # Modifica il numero di epoche come desiderato
#trainer.fit(t, train_loader)





INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
data = next(iter(fake_loader))
questions, answers = data
questions.shape